# Kütüphaneler

In [16]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from datetime import datetime, timedelta
import os

## Manuel Değiştirilebilir değişken: Ticker symbol

In [15]:
# Hedeflediğimiz borsa endeksini tanımlayan eşsiz kodu
ticker_symbol = '^DJI'
ticker = yf.Ticker(ticker_symbol)
borsa_adi=ticker.info['shortName']
print(f"Borsanın Orijinal Adı: {borsa_adi}")

Borsanın Orijinal Adı: Dow Jones Industrial Average


# Verileri İndir

'yfinance' kütüphanesi kullanarak Nasdaq (^IXIC) endeks verilerini 2010-01-01'den bugüne kadar indirme işlemi

In [10]:
today = datetime.today().strftime('%Y-%m-%d')
data = yf.download(ticker_symbol, start='2010-08-08', end=today)

# Borsa verilerini kontrol etmek için 10 Aralık 2024'e kadar olan fiyatları yazdır
print("10 Aralık 2024'e kadar olan borsa verileri:")
print(data.loc[:'2024-12-10'])

[*********************100%***********************]  1 of 1 completed

10 Aralık 2024'e kadar olan borsa verileri:
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2010-08-09  3764.250000  3781.629883  3752.989990  3777.370117  3777.370117   
2010-08-10  3769.469971  3771.669922  3710.010010  3730.580078  3730.580078   
2010-08-11  3705.439941  3708.620117  3625.699951  3628.290039  3628.290039   
2010-08-12  3611.459961  3641.850098  3584.439941  3621.070068  3621.070068   
2010-08-13  3650.459961  3652.080078  3577.540039  3610.909912  3610.909912   
...                 ...          ...          ...          ...          ...   
2024-12-04  7261.720215  7322.859863  7257.430176  7303.279785  7303.279785   
2024-12-05  7282.720215  7351.000000  7282.720215  7330.540039  7330.540039   
2024-12-06  7334.270020  7444.720215  7334.140137  7426.879883  7426.879883   
2024-12-09  7484.500000  7511.020020  7444.810059  7480.140137  7480.140137   
2024-12-

# Verileri Normalleştir

Kapanış fiyatlarını sütun vektörüne dönüştürüp 0-1 aralığında normalleştirir

In [11]:
close_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# Modeli Eğitim İçin Veri Hazırla

Veri setini 60 güne bölerek X (girdi) ve y (çıktı) çiftleri oluşturma işlemi

In [12]:
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(len(dataset) - time_step):
        X.append(dataset[i:(i + time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)


time_step = 60
X_train, y_train = create_dataset(scaled_data, time_step)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# LSTM Modeli Kurma ve Eğitme

50(LSTM) 50(LSTM) 25 1 katmanında nöron yapısı ile yapay zeka eğitme işlemi

In [13]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error') #Adaptive Moment Estimation
model.fit(X_train, y_train, batch_size=64, epochs=50)

Epoch 1/50


c:\Users\alper\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0288
Epoch 2/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 6.7410e-04
Epoch 3/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 6.7239e-04
Epoch 4/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 7.0447e-04
Epoch 5/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 5.7680e-04
Epoch 6/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 5.2614e-04
Epoch 7/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 5.5818e-04
Epoch 8/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 5.1131e-04
Epoch 9/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 4.9965e-04
Epoch 10/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.9837e-04
Epoch 11/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.8983e-04
Epoch 12/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.4214e-04
Epoch 13/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3.9351e-04
Epoch 14/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.4078e-04
Epoch 15/50
57/57 ━━━━━━━━━━━━

# Oluşan Sinir Ağını Kaydetme İşlemi

In [14]:
current_directory = os.getcwd() 
save_path = os.path.join(current_directory, ticker_symbol+".h5")

model.save(save_path)
print(f"Model başarıyla kaydedildi: {save_path}")

Model başarıyla kaydedildi: c:\Users\alper\Desktop\Ver 0.10\Ver 0.8\^FCHI.h5
